In [35]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('./nsl-kdd'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


./nsl-kdd\Attack_Types.csv
./nsl-kdd\d_loss.png
./nsl-kdd\g_loss.png
./nsl-kdd\KDDTest.csv
./nsl-kdd\KDDTrain.csv
./nsl-kdd\nslkdd_profile.html
./nsl-kdd\test.parquet
./nsl-kdd\train.parquet


In [36]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.autograd import grad
from tqdm import tqdm  # tqdm 用于显示进度条


import pickle, os
from collections import defaultdict
import seaborn as sns
sns.set_style("darkgrid")
# %matplotlib inline

import csv
import pandas as pd

# 检查是否有可用的 CUDA 设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [37]:
import pyarrow as pa
import pyarrow.parquet as pq


data_file_path = './nsl-kdd/'
df = pd.read_csv(data_file_path+'KDDTrain.csv')
# test = pd.read_csv(data_file_path+'KDDTest.csv')

In [38]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,dos
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [39]:
columns = df.columns

In [40]:
# missing_values = df.isnull().sum()
# print(f"Total missing values: {missing_values}")

# columns_with_missing_values = []

# for column, count in missing_values.items():
#     if count > 0:
#         columns_with_missing_values.append(column)

# print(f"Columns with missing values: {columns_with_missing_values}")

In [41]:
df  = df.dropna()


# label encoder

In [42]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder, RobustScaler
# 处理数据中的字符值
label_encoder = LabelEncoder()

# 
df['label'] = label_encoder.fit_transform(df['label'])


label_mapping = {l: i for i, l in enumerate(label_encoder.classes_)}
print(label_mapping)

col = ["protocol_type","service","flag"]

df[col] = OrdinalEncoder().fit_transform(df[col])


df.head()

{'dos': 0, 'normal': 1, 'probe': 2, 'r2l': 3, 'u2r': 4}


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,1.0,20.0,9.0,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,1
1,0,2.0,44.0,9.0,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,1
2,0,1.0,49.0,5.0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
3,0,1.0,24.0,9.0,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,1
4,0,1.0,24.0,9.0,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1


# data preprocessing

In [43]:
from sklearn.model_selection import train_test_split

to_drop = ['dst_host_srv_serror_rate','num_root','rerror_rate',
            'serror_rate','srv_rerror_rate','srv_serror_rate']
df.drop(to_drop,axis=1,inplace=True)


data_cols = list(df.columns[ df.columns != 'label' ])

scaler = RobustScaler(quantile_range=(0.1, 99.9))
df[data_cols] = scaler.fit_transform(df[data_cols])


In [44]:

# 将数据集分为训练集和测试集
x_train, x_test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

train, test = x_train, x_test

x_train.to_parquet('./nsl-kdd/train.parquet', engine='pyarrow')
x_test.to_parquet('./nsl-kdd/test.parquet', engine='pyarrow')


data_cols = list(x_train.columns[ x_train.columns != 'label' ])

y_train = x_train.label.values
y_test = x_test.label.values


att_ind = np.where(x_train.label != label_mapping["normal"])[0]
x = x_train[data_cols].values[att_ind]
y = y_train[att_ind].reshape(-1,1)
y

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [45]:
n = x_train.shape[1]
n

36

In [46]:
# generate noise for generator
def make_noise(batch_size):
    return torch.Tensor(np.random.normal(0,1/3,(batch_size,n))).to(device)

In [47]:
def plot_summary(d_l, g_l,acc_r,acc_g, m =''):
    n = np.arange(len(d_l))
    title = 'Loss and Accuracy plot'+'\n'+ m
    title = title.replace('.pickle','')
    fig, axs = plt.subplots(2,figsize=(19.20,10.80))

    axs[0].set_title(title,fontsize=20.0,fontweight="bold")
    axs[0].plot(n, g_l,label='Generator loss',linewidth=4)
    axs[0].plot(n, d_l,label='Discriminator loss',linewidth=4)
    axs[0].legend(loc=0, prop={'size': 20})
    axs[0].set_ylabel('Loss',fontsize=20.0,fontweight="bold")
    axs[0].tick_params(labelsize=20)
    axs[0].tick_params(axis='x',which='both',bottom=False,top=False,labelbottom=False,labelsize=20)

    # axs[1].plot(n, acc,'r',label='Discriminator accuracy',linewidth=4)
    axs[1].plot(n, acc_g,label='Accuracy on Generated',linewidth=4)
    axs[1].plot(n, acc_r,label='Accuracy on Real',linewidth=4)
    axs[1].legend(loc=0,prop={'size': 20})
    axs[1].set_ylabel('Accuracy',fontsize=20.0,fontweight="bold")
    axs[1].set_xlabel('Ephoc',fontsize=20.0,fontweight="bold")
    axs[1].tick_params(labelsize=20)

    plt.tight_layout()
    #plt.show()
    if not os.path.exists("imgs"):
        os.makedirs('imgs')
    plt.savefig(f'imgs/{m[:-7]}.png',dpi = 300)
    plt.close('all') #plt.close(fig)

# define the generator

In [48]:
from collections import defaultdict
import pickle, itertools

class CGAN(nn.Module):
    def __init__(self, arguments, X, y):
        """Conditional Generative Adversarial Network class"""
        super(CGAN, self).__init__()

        # Unpack the arguments
        [self.rand_noise_dim,self.n_layers, self.tot_epochs, self.batch_size, self.D_epochs, 
         self.G_epochs,  self.activation, 
         self.optimizer, self.learning_rate, self.min_num_neurons] = arguments

        # Convert data to tensors
        self.X_train = torch.tensor(X, dtype=torch.float32)
        self.y_train = torch.tensor(y, dtype=torch.float32)

        # Define data dimensions
        self.label_dim = y.shape[1]
        self.x_data_dim = X.shape[1]

        # Initialize tracking variables
        self.g_losses = []
        self.d_losses, self.disc_loss_real, self.disc_loss_generated = [], [], []
        self.acc_history = []
        
        # Define the models
        self.generator = self.build_generator()
        self.discriminator = self.build_discriminator()

        # Optimizers for Generator and Discriminator
        self.optim_G = optim.Adam(self.generator.parameters(), lr=self.learning_rate)
        self.optim_D = optim.Adam(self.discriminator.parameters(), lr=self.learning_rate)

        # Loss function
        self.criterion = nn.BCELoss()

        # For tracking purposes (naming the GAN based on arguments)
        self.gan_name = '_'.join(str(e) for e in arguments).replace(".", "")
        self.terminated = False

    def build_generator(self):
        """Create the generator model"""
        layers = []
        # Input layer: noise + labels
        layers.append(nn.Linear(self.rand_noise_dim + self.label_dim, self.min_num_neurons))
        layers.append(self.get_activation(self.activation))
        
        # Hidden layers
        for i in range(1, self.n_layers + 1):
            layers.append(nn.Linear(self.min_num_neurons * i, self.min_num_neurons * (i + 1)))
            layers.append(self.get_activation(self.activation))

        # Output layer: generate data
        layers.append(nn.Linear(self.min_num_neurons * (self.n_layers + 1), self.x_data_dim))
        layers.append(nn.Sigmoid())  # Normalize output to (0, 1)

        return nn.Sequential(*layers)

    def build_discriminator(self):
        """Create the discriminator model"""
        layers = []
        # Input layer: data + labels
        layers.append(nn.Linear(self.x_data_dim + self.label_dim, self.min_num_neurons * self.n_layers))
        layers.append(self.get_activation(self.activation))
        
        # Hidden layers (reverse order)
        input_dim = self.min_num_neurons * self.n_layers
        for n in reversed(range(1, self.n_layers + 1)):
            layers.append(nn.Linear(input_dim, self.min_num_neurons * n))
            layers.append(self.get_activation(self.activation))
            input_dim = self.min_num_neurons * n
        
        # Output layer: classify as real or fake
        layers.append(nn.Linear(self.min_num_neurons, 1))
        layers.append(nn.Sigmoid())  # Output is probability

        return nn.Sequential(*layers)

    def get_activation(self, activation_name):
        """Helper to get activation function"""
        if activation_name == 'relu':
            return nn.ReLU()
        elif activation_name == 'tanh':
            return nn.Tanh()
        elif activation_name == 'leakyrelu':
            return nn.LeakyReLU(0.2)
        elif activation_name == 'softplus':
            return nn.Softplus()  # 新增对 softplus 激活函数的支持
        elif activation_name == 'linear':
            return nn.Identity()  # linear 激活等同于不做任何变化，使用 nn.Identity()
        elif activation_name == 'elu':
            return nn.ELU()  # 新增对 ELU 激活函数的支持
        else:
            raise ValueError(f"Activation {activation_name} not implemented")

    def get_batch(self):
        """Get a random batch of training data"""
        batch_ix = np.random.choice(len(self.X_train), size=self.batch_size, replace=False)
        return self.X_train[batch_ix], self.y_train[batch_ix]
    
    def save_model(self,model_dir = "./models"):
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

         # 保存生成器
        # self.generator.save(f"{model_dir}/{self.gan_name}_generator_model.h5")
        torch.save(self.generator.state_dict(), f"{model_dir}/{self.gan_name}_generator_model.pth")

        # 保存判别器
        # self.discriminator.save(f"{model_dir}/{self.gan_name}_discriminator_model.h5")
        torch.save(self.discriminator.state_dict(), f"{model_dir}/{self.gan_name}_discriminator_model.pth")

        # 保存对抗模型（combined model）
        # self.combined.save(f"{model_dir}/{self.gan_name}_combined_model.h5")
        # torch.save(self.combined.state_dict(), f"{model_dir}/{self.gan_name}_combined_model.pth")

    def dump_to_file(self, save_dir="./logs"):
        """Save the training history to a file"""
        H = defaultdict(dict)
        H["acc_history"] = self.acc_history
        H["Generator_loss"] = self.g_losses
        H["disc_loss_real"] = self.disc_loss_real
        H["disc_loss_gen"] = self.disc_loss_generated
        H["discriminator_loss"] = self.d_losses
        H["rand_noise_dim"], H["total_epochs"] = self.rand_noise_dim, self.tot_epochs
        H["batch_size"], H["learning_rate"] = self.batch_size, self.learning_rate
        H["n_layers"], H["activation"] = self.n_layers, self.activation
        H["optimizer"], H["min_num_neurons"] = self.optimizer, self.min_num_neurons

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)


        with open(f"{save_dir}/CGAN_{self.gan_name}.pickle", "wb") as output_file:
            pickle.dump(H, output_file)

        with open(f"{save_dir}/CGAN_{self.gan_name}.pickle", 'rb') as f:
            x = pickle.load(f)

        d_l = np.array(x['discriminator_loss']).ravel()
        g_l = np.array(x['Generator_loss']).ravel()
        acc_history = np.array(x['acc_history'])
        acc = acc_history.sum(axis=1) * 0.5
        acc_real = acc_history[:,1]
        acc_gen = acc_history[:,0]

        filename = f"CGAN_{self.gan_name}.pickle"
        plot_summary(d_l, g_l,acc_real,acc_gen,filename)

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        with open(f"{save_dir}/{self.gan_name}{'.pickle'}", "wb") as output_file:
            pickle.dump(H,output_file)
        
        self.save_model()
        
            
    # 改进 labels 的生成
    def get_balanced_labels(self):
        """根据类别分布生成更多的少数类标签(3和4)"""
        labels = []
        for i in range(self.batch_size):
            # 假设 3 和 4 是少数类，提高它们生成的概率
            if np.random.rand() < 0.7:  # 设置少数类生成概率为 70%
                labels.append(np.random.choice([3, 4]))  # 在少数类中随机选择一个
            else:
                labels.append(np.random.choice([0, 1, 2]))  # 在多数类中选择一个
        return torch.tensor(labels, dtype=torch.float32)

    # 生成一些样本进行可视化
    def visualize_samples(self, num_samples=100):
        noise = torch.randn(num_samples, self.rand_noise_dim)
        labels = torch.randint(0, 5, (num_samples, self.label_dim), dtype=torch.float32)
        generated_samples = self.generator(torch.cat([noise, labels], 1))

        # 可视化生成的样本 (可根据特征进行降维后展示)
        plt.scatter(generated_samples[:, 0].detach().numpy(), generated_samples[:, 1].detach().numpy())
        plt.show()


    def train(self):
        """Train the CGAN model"""
        real_labels = torch.ones(self.batch_size, 1)
        fake_labels = torch.zeros(self.batch_size, 1)

        for epoch in range(self.tot_epochs):
            # Train Discriminator
            for i in range(self.D_epochs):
                x, labels = self.get_batch()
                noise = torch.randn(self.batch_size, self.rand_noise_dim)
                generated_x = self.generator(torch.cat([noise, labels], 1))

                # Train on real and fake data
                self.optim_D.zero_grad()
                real_loss = self.criterion(self.discriminator(torch.cat([x, labels], 1)), real_labels)
                fake_loss = self.criterion(self.discriminator(torch.cat([generated_x.detach(), labels], 1)), fake_labels)
                d_loss = 0.5 * (real_loss + fake_loss)
                d_loss.backward()
                self.optim_D.step()

            # Train Generator
            for j in range(self.G_epochs):
                self.optim_G.zero_grad()
                noise = torch.randn(self.batch_size, self.rand_noise_dim)
                labels = torch.randint(0, 5, (self.batch_size, self.label_dim), dtype=torch.float32)
                generated_x = self.generator(torch.cat([noise, labels], 1))
                g_loss = self.criterion(self.discriminator(torch.cat([generated_x, labels], 1)), real_labels)
                g_loss.backward()
                self.optim_G.step()

            # Record losses for analysis
            self.g_losses.append(g_loss.item())
            self.d_losses.append(d_loss.item())
            self.acc_history.append([real_loss.item(), fake_loss.item()])

            # print(f"Epoch {epoch} [D loss: {d_loss.item():.4f}, G loss: {g_loss.item():.4f}]")

            # Terminate if NaN is detected in loss
            if torch.isnan(d_loss) or torch.isnan(g_loss):
                self.terminated = True
                break


# set parameters


In [49]:
# rand_dim = np.arange(10,110,10)
# base_n_count = np.arange(3,41,3)
# ephocs = np.arange(100,5000,100)
# batch_sizes = [64,128,250,300,350]
# learning_rates = np.logspace(-1,-4,num=20)
# num_layers = np.arange(3,20)

# optimizers = ["sgd", "RMSprop", "adam", "Adagrad", "Adamax","Nadam"]
# activation_func = ["tanh","relu","softplus","linear","elu"]


base_n_count = [27] # 调整基本神经元数量的范围，从3-41变为5-51，步长为5
ephocs = [2000]  # 调整周期范围，从100-5000变为500-5500，步长为500
batch_sizes = [ 128]  # 简化批量大小选择为128, 256, 512
learning_rates =  [0.0005]  # 调整学习率范围，从0.01到0.00001，共10个值
num_layers = [4]  # 调整隐藏层数量，范围从2到10

optimizers = ["sgd"]  # 精简优化器选项，去除一些效果可能较差的
activation_func = ["relu"]  # 精简激活函数选项

d_epochs = [1]
g_epochs = [1]


#create a logs directory
if not os.path.exists('logs'):
    os.makedirs('logs')

In [50]:
# # 生成所有可能的参数组合
# tot = list(itertools.product([32],ephocs,batch_sizes,[1],[1],\
#                              learning_rates,num_layers,activation_func,optimizers,base_n_count))
# for i in tot:
#     args = list(i)
#     cgan = CGAN(args,x,y)
#     cgan.train()
#     if not cgan.terminated :
#         cgan.dump_to_file()

In [51]:

# 生成所有可能的参数组合
tot = list(itertools.product([32],num_layers,ephocs,batch_sizes,[1],[1],\
                            activation_func,optimizers,learning_rates,base_n_count))
# for i in tot:
#     args = list(i)
#     cgan = CGAN(args,x,y)
#     cgan.train()
#     if not cgan.terminated :
#         cgan.dump_to_file()


# 使用 tqdm 显示进度条
for idx, i in tqdm(enumerate(tot), total=len(tot), desc="Processing"):
    args = list(i)
    cgan = CGAN(args, x, y)  # 假设你已经定义了 CGAN 类
    cgan.train()  # 假设 train 方法存在
    if not cgan.terminated:
        cgan.dump_to_file()  # 假设 dump_to_file 方法存在



Processing: 100%|██████████| 1/1 [00:32<00:00, 32.02s/it]


# Analysis of the data generated by CGAN.

In [52]:
train,test, label_mapping = preprocessing.get_data(encoding="Label")
data_cols = list(train.columns[ train.columns != 'label' ])
train = preprocessing.normalize_data(train,data_cols)
test = preprocessing.normalize_data(test,data_cols)
x_train , x_test = preprocessing.preprocess(train,test,data_cols,"Robust",True)
# x_train = preprocessing.remove_outliers(train)

# train, test = None, None
y_train = x_train.label.values
y_test = x_test.label.values

NameError: name 'preprocessing' is not defined

In [ ]:
data_cols = list(x_train.columns[ x_train.columns != 'label' ])
to_drop = preprocessing.get_contant_featues(x_train,data_cols,threshold=0.995)
x_train.drop(to_drop, axis=1,inplace=True)
x_test.drop(to_drop, axis=1,inplace=True)
data_cols = list(x_train.columns[ x_train.columns != 'label' ])

print("Total data features : {}".format(len(data_cols)))

In [ ]:
# Read keras model
from keras.models import load_model
model_path = './models/generator_model.h5'
model = load_model(model_path)

In [ ]:
normal_df = x_train.query('label == 1')
print("# of samples : {}".format(normal_df.shape[0]))

In [ ]:
Q1 = normal_df[data_cols].quantile(0.25)
Q3 = normal_df[data_cols].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
swarm_plot = sns.pairplot(train.query('label != 1'),hue='label')
# fig = swarm_plot.get_figure()
plt.savefig("all.png",dpi = 300)

In [ ]:
dos_df = x_train.query('label == 0')
print("Number of samples : {}".format(dos_df.shape[0]))

In [ ]:
Q1 = dos_df[data_cols].quantile(0.25)
Q3 = dos_df[data_cols].quantile(0.75)
IQR = Q3 - Q1
# print(IQR)

In [ ]:
probe_df = x_train.query('label == 2')
print("Number of probe attack samples : {}".format(probe_df.shape[0]))

In [ ]:
Q1 = probe_df[data_cols].quantile(0.25)
Q3 = probe_df[data_cols].quantile(0.75)
IQR = Q3 - Q1
# print(IQR)

In [ ]:
#Plot the difference of fake and real attacks

figure(num=None, figsize=(10, 8), dpi=200, facecolor='w', edgecolor='k')

pca = PCA(n_components=2)
pca.fit(probe_df.drop("label",axis=1))
decomposed_probe_df = pca.transform(probe_df.drop("label",axis=1))

noise = np.random.normal(0, 1, (len(probe_df), 32))
generated_probe = model.predict([noise, probe_df.label.values.reshape(-1,1)])[:,:-1]

gen_decomposed = PCA(n_components=2).fit_transform(generated_probe)
plt.scatter(x=decomposed_probe_df[:,0],y=decomposed_probe_df[:,1],label="Real")
plt.scatter(x=gen_decomposed[:,0],y=gen_decomposed[:,1],label="Generated")
plt.title("Probe Real and Generated samples")
plt.legend()
plt.show()